In [1]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [5]:
import pandas as pd
import numpy as np
from random import sample, randint
import os, datetime
from pathlib import Path
import itertools
from sklearn.preprocessing import StandardScaler,MinMaxScaler
import logging
from datetime import timedelta
import datetime
import matplotlib.pyplot as plt
import matplotlib as mpl
import  warnings,sklearn
import pickle
# Import the email modules we'll need
from sklearn.base import BaseEstimator, ClassifierMixin
from email.message import EmailMessage
from sklearn.model_selection import train_test_split
from sklearn_pandas import DataFrameMapper
from sklearn.pipeline import make_pipeline, make_union
from tpot.builtins import StackingEstimator
from pandas.api.types import is_string_dtype, is_numeric_dtype
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.optimizers import Adam, Adadelta, SGD
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras import layers
from tensorflow.keras import Input
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras import regularizers
import random
from keras_multi_input import *


In [6]:
current_directory = '.'
log_file = os.path.join(current_directory, 'data/wp.log')
feather_file = os.path.join(current_directory, 'data/total_data_filled.feather')
df = pd.read_feather(feather_file)
df = df.set_index([ 'index', 'day'])
# read in the prepared data set
logger = logging.getLogger('wp')
logger.setLevel(logging.DEBUG)
fh = logging.FileHandler(log_file)
fh.setLevel(logging.INFO)
ch = logging.StreamHandler()
ch.setLevel(logging.DEBUG)
# create formatter and add it to the handlers
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
fh.setFormatter(formatter)
ch.setFormatter(formatter)
# add the handlers to the logger
logger.addHandler(fh)
logger.addHandler(ch)
# date_fields = ["temperatureMaxTime", "temperatureMinTime", "apparentTemperatureMinTime",
#                 "apparentTemperatureHighTime","sunsetTime", "uvIndexTime"  ,"sunriseTime","temperatureHighTime", "temperatureLowTime", 
#                  "apparentTemperatureMaxTime",
#                  "apparentTemperatureLowTime"]

# for date_field in date_fields:
#     name = date_field.replace('Time', 'Hour')
#     df[name] = df[date_field].apply(lambda x: x.hour)
df = df.drop([ 'energy_count', "temperatureMaxTime", "temperatureMinTime", "apparentTemperatureMinTime",
                "apparentTemperatureHighTime","sunsetTime", "uvIndexTime"  ,"sunriseTime","temperatureHighTime", "temperatureLowTime", 
                 "apparentTemperatureMaxTime",
                 "apparentTemperatureLowTime", 'year'], axis = 1)



In [7]:
df = df.drop([ 'energy_mean', 'energy_max', 'energy_std','energy_min', 
              'energy_skewness', 'energy_kurtosis', 
              'energy_median','energey_max_usage_hour'], 
              axis = 1)

logger.info(df.head())

validation_days = 15

2020-02-14 20:35:52,176 - wp - INFO -                           LCLid  energy_sum stdorToU    Acorn Acorn_grouped  \
index     day                                                                 
MAC000002 2012-10-12  MAC000002       7.098      Std  ACORN-A      Affluent   
          2012-10-13  MAC000002      11.087      Std  ACORN-A      Affluent   
          2012-10-14  MAC000002      13.223      Std  ACORN-A      Affluent   
          2012-10-15  MAC000002      10.257      Std  ACORN-A      Affluent   
          2012-10-16  MAC000002       9.769      Std  ACORN-A      Affluent   

                      temperatureMax  windBearing               icon  \
index     day                                                          
MAC000002 2012-10-12           11.53        252.0  partly-cloudy-day   
          2012-10-13           12.32        300.0  partly-cloudy-day   
          2012-10-14           13.03        220.0  partly-cloudy-day   
          2012-10-15           14.74        226.

In [8]:
categorical_columns = ['LCLid','Acorn', 'Acorn_grouped', "icon", "stdorToU", "Type", "day.of.week", 'precipType',  'summary', 'before_holiday', 'after_holiday', 'month']
non_categorical_columns =list(filter(lambda x: x not in categorical_columns, df.columns))

label_column = 'energy_sum'
logger.info(categorical_columns)

for v in categorical_columns:
    df[v] = df[v].astype('category').cat.as_ordered()
for v in non_categorical_columns:
    df[v] = df[v].astype('float32')
cat_sz = [(c, len(df[c].cat.categories)+1) for c in categorical_columns]
emb_szs = [(c, min(50, (c+1)//2)) for _,c in cat_sz]
logger.info(emb_szs)

2020-02-14 20:35:57,504 - wp - INFO - ['LCLid', 'Acorn', 'Acorn_grouped', 'icon', 'stdorToU', 'Type', 'day.of.week', 'precipType', 'summary', 'before_holiday', 'after_holiday', 'month']
2020-02-14 20:36:04,016 - wp - INFO - [(5567, 50), (20, 10), (6, 3), (7, 4), (3, 2), (13, 7), (8, 4), (3, 2), (87, 44), (7, 4), (7, 4), (13, 7)]


In [9]:
df, y, nas, mapper = proc_df(df, y_fld=label_column,  do_scale=True)
non_categorical_columns =list(filter(lambda x: x not in categorical_columns, df.columns))
df = df[categorical_columns + non_categorical_columns ]
logger.info(df.dtypes)


2020-02-14 20:36:36,825 - wp - INFO - LCLid                            int16
Acorn                             int8
Acorn_grouped                     int8
icon                              int8
stdorToU                          int8
Type                              int8
day.of.week                       int8
precipType                        int8
summary                           int8
before_holiday                    int8
after_holiday                     int8
month                             int8
temperatureMax                 float64
windBearing                    float64
dewPoint                       float64
cloudCover                     float64
windSpeed                      float64
pressure                       float64
apparentTemperatureHigh        float64
visibility                     float64
humidity                       float64
apparentTemperatureLow         float64
apparentTemperatureMax         float64
uvIndex                        float64
temperatureLow            

In [10]:
logger.info(df.head())

2020-02-14 20:36:36,852 - wp - INFO -                       LCLid  Acorn  Acorn_grouped  icon  stdorToU  Type  \
index     day                                                             
MAC000002 2012-10-12      1      2              4     4         1     8   
          2012-10-13      1      2              4     4         1     8   
          2012-10-14      1      2              4     4         1     8   
          2012-10-15      1      2              4     6         1     8   
          2012-10-16      1      2              4     4         1     8   

                      day.of.week  precipType  summary  before_holiday  ...  \
index     day                                                           ...   
MAC000002 2012-10-12            1           1       84               6  ...   
          2012-10-13            3           1       65               6  ...   
          2012-10-14            4           1       74               6  ...   
          2012-10-15            2        

In [11]:
logger.info(df.columns)

2020-02-14 20:36:36,883 - wp - INFO - Index(['LCLid', 'Acorn', 'Acorn_grouped', 'icon', 'stdorToU', 'Type',
       'day.of.week', 'precipType', 'summary', 'before_holiday',
       'after_holiday', 'month', 'temperatureMax', 'windBearing', 'dewPoint',
       'cloudCover', 'windSpeed', 'pressure', 'apparentTemperatureHigh',
       'visibility', 'humidity', 'apparentTemperatureLow',
       'apparentTemperatureMax', 'uvIndex', 'temperatureLow', 'temperatureMin',
       'temperatureHigh', 'apparentTemperatureMin', 'moonPhase',
       'temperature_skewness', 'temperature_kurtosis', 'day_length',
       'temperatureMaxHour', 'temperatureMinHour',
       'apparentTemperatureMinHour', 'apparentTemperatureHighHour',
       'sunsetHour', 'uvIndexHour', 'sunriseHour', 'temperatureHighHour',
       'temperatureLowHour', 'apparentTemperatureMaxHour',
       'apparentTemperatureLowHour'],
      dtype='object')


In [12]:
max_date = max(df.index)[1]
valid_start_day = max_date - timedelta(days=validation_days) 

train_idx = list(filter(lambda t: t[1] < valid_start_day, df.index))
valid_idx = list(filter(lambda t: t[1] >= valid_start_day, df.index))
max_y = max(y)*1.1
y = pd.DataFrame(y, index = df.index, columns=['Value'])
X_train = df.loc[train_idx]
y_train = y.loc[train_idx].values.reshape(-1,1)
X_valid = df.loc[valid_idx]
y_valid = y.loc[valid_idx].values.reshape(-1,1)
y_train = y_train/max_y
y_valid = y_valid/max_y
    

In [13]:
#model setup
batch_size=200
epochs=20
lr = 0.01
l2_reg= 0.0

cat_input = [Input(shape=(1,), dtype='int32', name=c) for c in categorical_columns]
all_layers = []
for i in range(len(cat_input)):
    emb = layers.Embedding(emb_szs[i][0], emb_szs[i][1])(cat_input[i])
    flat = layers.Flatten()(emb)
    all_layers.append(flat)

contInput = Input(shape=(len(non_categorical_columns),), dtype='float32', name='continuouse')
all_layers.append(contInput)
lay = layers.concatenate(all_layers, axis =-1)

#lay = BatchNormalization()(lay)
lay = Dense(64, activation='relu', kernel_regularizer=regularizers.l2(l2_reg))(lay)
lay = BatchNormalization()(lay)
lay = Dense(16, activation='relu', kernel_regularizer=regularizers.l2(l2_reg))(lay)
lay = BatchNormalization()(lay)
answer = layers.Dense(1, activation='sigmoid')(lay)
inputs_all = cat_input
inputs_all.append(contInput)
model = Model(inputs_all, answer)
logger.info(model.summary())
adam = Adam(lr=lr)
model.compile(loss='binary_crossentropy', optimizer=adam,metrics=['mse', 'mae'])

2020-02-14 20:38:36,568 - wp - INFO - None


Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
LCLid (InputLayer)              [(None, 1)]          0                                            
__________________________________________________________________________________________________
Acorn (InputLayer)              [(None, 1)]          0                                            
__________________________________________________________________________________________________
Acorn_grouped (InputLayer)      [(None, 1)]          0                                            
__________________________________________________________________________________________________
icon (InputLayer)               [(None, 1)]          0                                            
______________________________________________________________________________________________

In [21]:
#fit data
batch_size = 200
epochs=20
name = './data/separate_LcLid.hdf5'
checkpointer = ModelCheckpoint(filepath=name, verbose=2, save_best_only=True)
train_values = {c:X_train[c] for c in categorical_columns}
train_values['continuouse'] = X_train[non_categorical_columns]

val_values = {c:X_valid[c] for c in categorical_columns}
val_values['continuouse'] = X_valid[non_categorical_columns]

history = model.fit(train_values, y_train, epochs=epochs, batch_size = batch_size,
                             validation_data=(val_values, y_valid), 
                             verbose=2, shuffle=True,
                             #validation_split=0.2,
                             callbacks=[checkpointer])

Train on 6904877 samples, validate on 152624 samples
Epoch 1/20
 - 399s - loss: nan - mean_squared_error: nan - mean_absolute_error: nan - val_loss: 15.6909 - val_mean_squared_error: nan - val_mean_absolute_error: nan

Epoch 00001: val_loss improved from inf to 15.69089, saving model to ./data/separate_LcLid.hdf5
Epoch 2/20


KeyboardInterrupt: 

In [ ]:
fig, ax = plt.subplots(figsize=(10, 5))
ax.plot(history.history['loss'], label='train')
ax.plot(history.history['val_loss'], label='test')
ax.legend()

In [ ]:
print(y_train[:10])

In [18]:
print(max(y_valid))

[0.39791518]
